In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "/content/drive/MyDrive/Colab Notebooks/platform_subscribe/"

train = pd.read_csv(base_path + "train.csv")
test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "sample_submission.csv")

In [39]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.1 MB/s eta 0:00:00


In [40]:
# 1. Linear Classifier
from sklearn.linear_model import SGDClassifier
# 2. Logistic Regression
from sklearn.linear_model import LogisticRegression
# 3. Decision Tree
from sklearn.tree import DecisionTreeClassifier
# 4. Random Forest
from sklearn.ensemble import RandomForestClassifier
# 5. CatBoost
from catboost import CatBoostClassifier


# 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

evulation_metric = f1_score

In [41]:
train_ONE = pd.get_dummies(train, columns=['preferred_difficulty_level', 'subscription_type'])

In [42]:
X=train_ONE.drop(columns=['user_id','target'])
y=train_ONE.target

In [43]:
from sklearn.model_selection import train_test_split
# 트테트테
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)

# 6:2:2 = train: validation:test
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(6000, 16) (2000, 16) (2000, 16) (6000,) (2000,) (2000,)


### data_scaling만 진행


In [44]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_val_scaled = std.transform(X_val)
X_train_scaled
X_val_scaled

array([[ 1.66696644e-01, -2.40051804e-01,  6.35886219e-01, ...,
         1.52449983e+00, -1.22602158e+00,  1.22602158e+00],
       [ 1.07677841e+00,  1.31730000e+00,  5.23133399e-01, ...,
         1.52449983e+00, -1.22602158e+00,  1.22602158e+00],
       [-8.95065419e-01,  1.19337073e-01, -7.65554775e-01, ...,
         1.52449983e+00,  8.15646329e-01, -8.15646329e-01],
       ...,
       [-1.50178660e+00, -4.79644389e-01,  2.52417230e+00, ...,
        -6.55952846e-01,  8.15646329e-01, -8.15646329e-01],
       [ 1.68349959e+00, -5.99440682e-01,  8.55214002e-01, ...,
        -6.55952846e-01, -1.22602158e+00,  1.22602158e+00],
       [-7.43385124e-01, -4.59219122e-04,  1.79950299e-01, ...,
         1.52449983e+00, -1.22602158e+00,  1.22602158e+00]])

In [45]:
clf_dt = DecisionTreeClassifier()
clf_rf = RandomForestClassifier()
clf_cb = CatBoostClassifier()

clf_dt.fit(X_train_scaled,y_train)
clf_rf.fit(X_train_scaled,y_train)
clf_cb.fit(X_train_scaled,y_train)

pred_val_dt=clf_dt.predict(X_val_scaled)
pred_val_rf=clf_rf.predict(X_val_scaled)
pred_val_cv=clf_cb.predict(X_val_scaled)

print('Desicion Tree : ' , evulation_metric(y_val, pred_val_dt))
print(pred_val_dt)
print('Random Forest : ' , evulation_metric(y_val, pred_val_rf))
print(pred_val_rf)
print('Catboost : ' , evulation_metric(y_val, pred_val_cv))
print(pred_val_cv)

Learning rate set to 0.022141
0:	learn: 0.6901623	total: 50ms	remaining: 50s
1:	learn: 0.6872536	total: 56.9ms	remaining: 28.4s
2:	learn: 0.6846855	total: 65.4ms	remaining: 21.7s
3:	learn: 0.6827959	total: 71.8ms	remaining: 17.9s
4:	learn: 0.6807561	total: 78ms	remaining: 15.5s
5:	learn: 0.6788517	total: 86.2ms	remaining: 14.3s
6:	learn: 0.6774039	total: 94ms	remaining: 13.3s
7:	learn: 0.6754255	total: 102ms	remaining: 12.6s
8:	learn: 0.6737711	total: 110ms	remaining: 12.1s
9:	learn: 0.6724925	total: 118ms	remaining: 11.7s
10:	learn: 0.6712316	total: 126ms	remaining: 11.3s
11:	learn: 0.6691783	total: 134ms	remaining: 11s
12:	learn: 0.6679384	total: 137ms	remaining: 10.4s
13:	learn: 0.6662401	total: 141ms	remaining: 9.94s
14:	learn: 0.6653882	total: 144ms	remaining: 9.47s
15:	learn: 0.6646715	total: 148ms	remaining: 9.07s
16:	learn: 0.6639416	total: 160ms	remaining: 9.25s
17:	learn: 0.6625466	total: 163ms	remaining: 8.9s
18:	learn: 0.6619167	total: 167ms	remaining: 8.62s
19:	learn: 0.66

### random forest가 decision tree보다 성능이 좋음

#### X_test 확인

In [46]:
X_test_scaled = std.transform(X_test)
X_test_scaled

array([[-0.28834424,  0.35892966,  0.2408262 , ..., -0.65595285,
         0.81564633, -0.81564633],
       [ 0.47005723,  0.23913337, -0.38008139, ...,  1.52449983,
         0.81564633, -0.81564633],
       [-1.04674571, -0.47964439, -0.26540216, ..., -0.65595285,
        -1.22602158,  1.22602158],
       ...,
       [-1.5017866 ,  1.07770741, -0.5779501 , ...,  1.52449983,
        -1.22602158,  1.22602158],
       [-1.19842601, -0.59944068,  2.04490767, ...,  1.52449983,
         0.81564633, -0.81564633],
       [ 1.68349959,  0.95791112, -0.10698085, ...,  1.52449983,
         0.81564633, -0.81564633]])

In [47]:
pred_test_rf=clf_rf.predict(X_test_scaled)
pred_test_cb=clf_cb.predict(X_test_scaled)

print('Random Forest : ' , evulation_metric(y_test, pred_test_rf))
print(pred_test_rf)
print('Catboost : ' , evulation_metric(y_val, pred_test_cb))
print(pred_test_cb)

Random Forest :  0.7384007897334649
[1 1 1 ... 1 1 1]
Catboost :  0.7409343351845802
[1 1 1 ... 1 0 1]


#### submission


In [34]:
X_submission=pd.get_dummies(test, columns=['preferred_difficulty_level', 'subscription_type'])
X_submission=X_submission.drop(columns=['user_id'])
X_submission_scaled = std.transform(X_submission)

In [35]:
preds=clf_rf.predict(X_submission_scaled)
preds

array([1, 1, 0, ..., 1, 0, 1])

In [36]:
submission['target']=preds
submission

,user_id,target
0,0001d6e9,1
1,0002c77d,1
2,0002df5b,0
3,000b6068,1
4,00184a0c,1
...,...,...
9995,ffe2eba5,1
9996,ffe710f1,1
9997,ffeccdef,1
9998,fff3fcea,0


In [37]:
submission.to_csv(base_path+"submission_1209_2.csv", index=False)